In [6]:
!pip install selenium

In [408]:
# all imports

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import json
import time

import re 
import pandas as pd

from multiprocessing import Pool

import os

In [745]:
#URL CENTRAL
#GENERAL URL: 'https://airbnb.com'
airbnb_urlL= 'https://www.airbnb.com/s/New-York--New-York--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=calendar&query=New%20York%2C%20New%20York%2C%20United%20States&place_id=ChIJOwg_06VPwokRYv534QaPC8g&source=structured_search_input_header&search_type=autocomplete_click'

# START OF USEFUL CODE 

In [746]:
b_soup = BeautifulSoup(requests.get(airbnb_url).content, 'html.parser')

In [747]:
print(b_soup.prettify())

<!DOCTYPE html>
<html data-is-hyperloop="true" dir="ltr" lang="en">
 <meta charset="utf-8"/>
 <meta content="en" name="locale"/>
 <meta content="notranslate" name="google"/>
 <meta content="authenticity_token" id="csrf-param-meta-tag" name="csrf-param"/>
 <meta content="" id="csrf-token-meta-tag" name="csrf-token"/>
 <meta content="" id="english-canonical-url"/>
 <meta content="on" name="twitter:widgets:csp"/>
 <meta content="yes" name="mobile-web-app-capable"/>
 <meta content="yes" name="apple-mobile-web-app-capable"/>
 <meta content="Airbnb" name="application-name"/>
 <meta content="Airbnb" name="apple-mobile-web-app-title"/>
 <meta content="#ffffff" name="theme-color"/>
 <meta content="#ffffff" name="msapplication-navbutton-color"/>
 <meta content="black-translucent" name="apple-mobile-web-app-status-bar-style"/>
 <meta content="/?utm_source=homescreen" name="msapplication-starturl"/>
 <script>
  (function() {
  var pgRequest = new XMLHttpRequest();
  var diffStamp = Date.now().toSt

In [748]:
listings = b_soup.find_all('div','_gig1e7')

In [749]:
listings

[<div class="_gig1e7"><div class="c4mnd7m dir dir-ltr"><div class="c1pbo4kt cikyho8 dir dir-ltr" style="--transition-element_transition-delay:0ms;--transition-element_transition-duration:200ms"><div class="_11ry7lz" data-testid="shimmer-legacy-listing-section-item"><div class="_o1549l" style="margin-top:12px;margin-bottom:24px"></div><div class="c1k4n2ms dir dir-ltr"><div class="mertcch dir dir-ltr"><span aria-busy="true" style="display:block;height:100%;width:100%;background-color:white;border-radius:var(--i-g-gvoq)"><span class="_8kplbn" style="animation-play-state:running;height:100%;width:100%;border-radius:var(--i-g-gvoq)"></span></span></div><div class="iabnva0 dir dir-ltr"><div style="margin-bottom:4px"><span aria-busy="true" style="display:block;height:18px;width:40%;background-color:white;border-radius:10px"><span class="_8kplbn" style="animation-play-state:running;height:100%;width:100%;border-radius:10px"></span></span></div><span aria-busy="true" style="display:block;height

In [16]:
print(listings[0].prettify())

<div class="_gig1e7">
 <div class="c4mnd7m dir dir-ltr">
  <div class="c1pbo4kt dir dir-ltr" style="--transition-element_transition-delay:0ms;--transition-element_transition-duration:200ms">
   <div class="_11ry7lz" data-testid="shimmer-legacy-listing-section-item">
    <div class="_o1549l" style="margin-top:12px;margin-bottom:24px">
    </div>
    <div class="c1k4n2ms dir dir-ltr">
     <div class="mertcch dir dir-ltr">
      <span aria-busy="true" style="display:block;height:100%;width:100%;background-color:white;border-radius:var(--i-g-gvoq)">
       <span class="_8kplbn" style="animation-play-state:running;height:100%;width:100%;border-radius:var(--i-g-gvoq)">
       </span>
      </span>
     </div>
     <div class="iabnva0 dir dir-ltr">
      <div style="margin-bottom:4px">
       <span aria-busy="true" style="display:block;height:18px;width:40%;background-color:white;border-radius:10px">
        <span class="_8kplbn" style="animation-play-state:running;height:100%;width:100%;bor

# TAGS FOR SEARCHING:
Price: 'span', 'a8jt5op dir dir-ltr' 

Title/Name: 'span', 'ts5gl90 tl3qa0j t1nzedvd dir dir-ltr'

Location: 'div', "mj1p6c8 dir dir-ltr"

Ratings: 'div', "sglmc5a dir dir-ltr"

Amentieis: 'div', "mp2hv9t dir dir-ltr"







In [ ]:
listings[8].find('div', "mj1p6c8 dir dir-ltr").get_text()

'Entire home in Flatbush'

In [ ]:
listings[6].find_all('span', "mp2hv9t dir dir-ltr")

[<span class="mp2hv9t dir dir-ltr">2 guests</span>,
 <span class="mp2hv9t dir dir-ltr">Studio</span>,
 <span class="mp2hv9t dir dir-ltr">1 bed</span>,
 <span class="mp2hv9t dir dir-ltr">1 bath</span>,
 <span class="mp2hv9t dir dir-ltr">Wifi</span>,
 <span class="mp2hv9t dir dir-ltr">Kitchen</span>,
 <span class="mp2hv9t dir dir-ltr">Air conditioning</span>]

# START OF FUNCTIONS

In [750]:
def scrape_page(page_url):
    """Extracts HTML from a webpage"""
    
    answer = requests.get(page_url)
    content = answer.content
    soup = BeautifulSoup(content, features='html.parser')
    
    return soup

In [751]:
def extract_listing(page_url):
    """Extracts listings from an Airbnb search page"""
    
    page_soup = scrape_page(page_url)
    listings = page_soup.findAll("div", {"class": "_gig1e7"})

    return listings

In [752]:
def extract_element_data(soup, params):
    """Extracts data from a specified HTML element"""
    
    # 1. Find the right tag
    if 'class' in params:
        elements_found = soup.find_all(params['tag'], params['class'])
    else:
        elements_found = soup.find_all(params['tag'])
        
    # 2. Extract text from these tags
    if 'get' in params:
        element_texts = [el.get(params['get']) for el in elements_found]
    else:
        element_texts = [el.get_text() for el in elements_found]
        
    # 3. Select a particular text or concatenate all of them
    tag_order = params.get('order', 0)
    if tag_order == -1:
        output = '**__**'.join(element_texts)
    else:
        output = element_texts[tag_order]
    
    return output

In [753]:
DATA_SEARCH = {
      'Listing_id': {'tag': 'a', 'get': 'target'},
      'Name': { 'tag': 'span', 'class': 'ts5gl90 tl3qa0j t1nzedvd dir dir-ltr'},
      'Price': {'tag': 'span', 'class': 'a8jt5op dir dir-ltr'},
      'Amenities': {'tag':'span', 'class': 'mp2hv9t dir dir-ltr', 'order': -1},
      'Ratings': {'tag': 'div', 'class': "sglmc5a dir dir-ltr", 'order': -1},
      'Location': {'tag': 'div', 'class': "mj1p6c8 dir dir-ltr"},
      'Home_URL': {'tag': 'a', 'get': 'href'}
      }

In [755]:
def extract_page_features(soup, rules):
    features_dict = {}
    for feature in rules:
        try:
            features_dict[feature] = extract_element_data(soup, rules[feature])
        except:
            features_dict[feature] = 'empty'
    
    return features_dict

In [756]:
def get_listing_data(airbnb_url, rules) :

    listing_soups = extract_listing(airbnb_url)

    features_list = pd.DataFrame()
    for listing in listing_soups:
        features_dict = {}

        for feature in DATA_SEARCH:
            features_dict[feature] = extract_element_data(listing, rules[feature])
            features_list['Full_URL'] = airbnb_url
            features_list = features_list.append(features_dict, ignore_index=True)

    return features_list

In [757]:
general_df = get_listing_data(airbnb_url, DATA_SEARCH)

In [759]:
general_df

,Full_URL,Listing_id,Name,Price,Amenities,Ratings,Location,Home_URL
0,https://www.airbnb.com/s/New-York--New-York--U...,listing_54163912,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.airbnb.com/s/New-York--New-York--U...,listing_54163912,Beautiful One Bedroom with Fantastic Views,NaN,NaN,NaN,NaN,NaN
2,https://www.airbnb.com/s/New-York--New-York--U...,listing_54163912,Beautiful One Bedroom with Fantastic Views,$72 per night,NaN,NaN,NaN,NaN
3,https://www.airbnb.com/s/New-York--New-York--U...,listing_54163912,Beautiful One Bedroom with Fantastic Views,$72 per night,2 guests**__**1 bedroom**__**1 bed**__**1 bath...,NaN,NaN,NaN
4,https://www.airbnb.com/s/New-York--New-York--U...,listing_54163912,Beautiful One Bedroom with Fantastic Views,$72 per night,2 guests**__**1 bedroom**__**1 bed**__**1 bath...,,NaN,NaN
...,...,...,...,...,...,...,...,...
135,https://www.airbnb.com/s/New-York--New-York--U...,listing_31543246,Brownstone Top Floor Sunny & Spacious Apartment,$90 per night,NaN,NaN,NaN,NaN
136,https://www.airbnb.com/s/New-York--New-York--U...,listing_31543246,Brownstone Top Floor Sunny & Spacious Apartment,$90 per night,3 guests**__**1 bedroom**__**2 beds**__**1 bat...,NaN,NaN,NaN
137,https://www.airbnb.com/s/New-York--New-York--U...,listing_31543246,Brownstone Top Floor Sunny & Spacious Apartment,$90 per night,3 guests**__**1 bedroom**__**2 beds**__**1 bat...,4.75 (8 reviews),NaN,NaN
138,https://www.airbnb.com/s/New-York--New-York--U...,listing_31543246,Brownstone Top Floor Sunny & Spacious Apartment,$90 per night,3 guests**__**1 bedroom**__**2 beds**__**1 bat...,4.75 (8 reviews),Entire rental unit in Harlem,NaN


In [760]:
def build_urls(url, listings_per_page=20, pages_per_location=15):
    """Builds links for all search pages for a given location"""
    
    #Since single list, no need for DataFrame
    url_list = []
    for i in range(pages_per_location):
        offset = listings_per_page * i
        url_pagination = url + f'&items_offset={offset}'
        url_list.append(url_pagination)
       
    return url_list

In [762]:
url_list = build_urls(airbnb_url)

In [763]:
url_list

['https://www.airbnb.com/s/New-York--New-York--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=calendar&place_id=ChIJOwg_06VPwokRYv534QaPC8g&query=New%20York%2C%20New%20York%2C%20United%20States&adults=1&search_type=filter_change&room_types%5B%5D=Entire%20home%2Fapt&items_offset=0',
 'https://www.airbnb.com/s/New-York--New-York--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=calendar&place_id=ChIJOwg_06VPwokRYv534QaPC8g&query=New%20York%2C%20New%20York%2C%20United%20States&adults=1&search_type=filter_change&room_types%5B%5D=Entire%20home%2Fapt&items_offset=20',
 'https://www.airbnb.com/s/New-York--New-York--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flex

In [764]:
url_list[1]

'https://www.airbnb.com/s/New-York--New-York--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=calendar&place_id=ChIJOwg_06VPwokRYv534QaPC8g&query=New%20York%2C%20New%20York%2C%20United%20States&adults=1&search_type=filter_change&room_types%5B%5D=Entire%20home%2Fapt&items_offset=20'

In [765]:
def process_search_pages(url_list, rules):
    features_list = []
    for page in url_list:
        listings = extract_listing(page)
        for listing in listings:
            features = extract_page_features(listing, rules)
            features_list.append(features)

    return features_list

In [766]:
process_search_pages(url_list, DATA_SEARCH)

[{'Listing_id': 'listing_54163912',
  'Name': 'Beautiful One Bedroom with Fantastic Views',
  'Price': '$72 per night',
  'Amenities': '2 guests**__**1 bedroom**__**1 bed**__**1 bath**__**Wifi**__**Kitchen**__**Self check-in**__**Air conditioning',
  'Ratings': '',
  'Location': 'Entire rental unit in Washington Heights',
  'Home_URL': '/rooms/54163912?adults=1&previous_page_section_name=1000&federated_search_id=2cd6d47d-cfdd-4870-9f11-edfec57853f8'},
 {'Listing_id': 'listing_580404909768914699',
  'Name': 'East Village with private roof July 2022 (plus)',
  'Price': '$75 per night',
  'Amenities': '2 guests**__**1 bedroom**__**1 bed**__**1 bath**__**Wifi**__**Kitchen**__**Air conditioning',
  'Ratings': '',
  'Location': 'Entire condo in East Village',
  'Home_URL': '/rooms/580404909768914699?adults=1&previous_page_section_name=1000&federated_search_id=2cd6d47d-cfdd-4870-9f11-edfec57853f8'},
 {'Listing_id': 'listing_48253767',
  'Name': 'Charming Astoria studio on quiet tree-lined str

In [767]:
def get_all_listings_data_300(url):
    all_pages_url = build_urls(url)

    all_data = pd.DataFrame()
    for individual_url in all_pages_url:
        all_data = all_data.append(get_listing_data(individual_url, DATA_SEARCH), ignore_index=True)
    return all_data





In [768]:
ny_gen_300 = get_all_listings_data_300(airbnb_url)

In [769]:
ny_gen_300

,Full_URL,Listing_id,Name,Price,Amenities,Ratings,Location,Home_URL
0,https://www.airbnb.com/s/New-York--New-York--U...,listing_580404909768914699,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.airbnb.com/s/New-York--New-York--U...,listing_580404909768914699,East Village with private roof July 2022 (plus),NaN,NaN,NaN,NaN,NaN
2,https://www.airbnb.com/s/New-York--New-York--U...,listing_580404909768914699,East Village with private roof July 2022 (plus),$75 per night,NaN,NaN,NaN,NaN
3,https://www.airbnb.com/s/New-York--New-York--U...,listing_580404909768914699,East Village with private roof July 2022 (plus),$75 per night,2 guests**__**1 bedroom**__**1 bed**__**1 bath...,NaN,NaN,NaN
4,https://www.airbnb.com/s/New-York--New-York--U...,listing_580404909768914699,East Village with private roof July 2022 (plus),$75 per night,2 guests**__**1 bedroom**__**1 bed**__**1 bath...,,NaN,NaN
...,...,...,...,...,...,...,...,...
2095,https://www.airbnb.com/s/New-York--New-York--U...,listing_34315737,Studio Apartment in BoroPark Brooklyn NY,$43 per night,NaN,NaN,NaN,NaN
2096,https://www.airbnb.com/s/New-York--New-York--U...,listing_34315737,Studio Apartment in BoroPark Brooklyn NY,$43 per night,2 guests**__**1 bedroom**__**2 beds**__**1 bat...,NaN,NaN,NaN
2097,https://www.airbnb.com/s/New-York--New-York--U...,listing_34315737,Studio Apartment in BoroPark Brooklyn NY,$43 per night,2 guests**__**1 bedroom**__**2 beds**__**1 bat...,4.0 (20 reviews),NaN,NaN
2098,https://www.airbnb.com/s/New-York--New-York--U...,listing_34315737,Studio Apartment in BoroPark Brooklyn NY,$43 per night,2 guests**__**1 bedroom**__**2 beds**__**1 bat...,4.0 (20 reviews),Entire guest suite in Southwestern Brooklyn,NaN


# GOING INTO LISTING DETAILS

In [79]:
answer = requests.get('https://www.airbnb.com/rooms/50616407')
detail_soup = BeautifulSoup(answer.content)

In [1]:
conda install selenium

Solving environment: done

## Package Plan ##

  environment location: /Users/Vero/opt/anaconda3

  added / updated specs:
    - selenium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.12.0               |   py38hecd8cb5_0        14.5 MB
    selenium-3.141.0           |py38h1de35cc_1001         812 KB
    ------------------------------------------------------------
                                           Total:        15.3 MB

The following NEW packages will be INSTALLED:

  selenium           pkgs/main/osx-64::selenium-3.141.0-py38h1de35cc_1001

The following packages will be UPDATED:

  conda                               4.10.3-py38hecd8cb5_0 --> 4.12.0-py38hecd8cb5_0



conda-4.12.0         | 14.5 MB   | ##################################### | 100% 
selenium-3.141.0     | 812 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying tra

In [186]:
#BROOKLYN URL 
detail_url='https://www.airbnb.com/rooms/50616407'

In [771]:

driver = webdriver.Chrome('/Users/Vero/Downloads/chromedriver')
driver.get(detail_url)

In [188]:
detail_soup

<html class="scrollbar-gutter js-focus-visible" data-is-hyperloop="true" dir="ltr" lang="en" style="--vh:6.6px; --vw:12px; --vw-unitless:1200; --vw-px:1200px;"><head><meta charset="utf-8"/><meta content="en" name="locale"/><meta content="notranslate" name="google"/><meta content="authenticity_token" id="csrf-param-meta-tag" name="csrf-param"/><meta content="V4$.airbnb.com$4qaPBRBvDXk$x6ZjGVp-961KPlDDdfM6Lnvq37vAgpEAttDA5QpHDh4=" id="csrf-token-meta-tag" name="csrf-token"/><meta content="" id="english-canonical-url"/><meta content="on" name="twitter:widgets:csp"/><meta content="yes" name="mobile-web-app-capable"/><meta content="yes" name="apple-mobile-web-app-capable"/><meta content="Airbnb" name="application-name"/><meta content="Airbnb" name="apple-mobile-web-app-title"/><meta content="#ffffff" name="theme-color"/><meta content="#ffffff" name="msapplication-navbutton-color"/><meta content="black-translucent" name="apple-mobile-web-app-status-bar-style"/><meta content="/?utm_source=hom

In [772]:
page_detailed = driver.page_source

In [773]:
driver.quit()
detail_soup = BeautifulSoup(page_detailed)

In [775]:
detail_soup.find('span', "ll4r2nl dir dir-ltr")

In [776]:
options = Options()
options.add_argument('--blink-settings=imagesEnabled=false')
driver = webdriver.Chrome('/Users/Vero/Downloads/chromedriver', options=options)
driver.get(detail_url)

In [777]:
#TRYING OUT REVIEWS BUTTON
driver = webdriver.Chrome('/Users/Vero/Downloads/chromedriver')

driver.get(detail_url)
time.sleep(10)
element = driver.find_element_by_class_name('_koyt9le')
element.click()
page_detailed = driver.page_source
driver.quit()
detail_soup_clicked = BeautifulSoup(page_detailed)
# driver.find_element_by_xpath("//button[contains(@class='_9n9cos7')]")
#driver.find_element_by_class_name('_le6wlg').click()
#driver.find_element_by_css_selector('[data-testid="homes-pdp-cta-btn"]')

NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=100.0.4896.127)


In [216]:
#DESCRIPTION TEXT
detail_soup.find('span', "ll4r2nl dir dir-ltr").get_text()

'Located in the heart of Brooklyn\'s "Little Haiti," this 1,000 sq ft 3 bedroom/2bath comes with all the amenities! Newly renovated, with splits (controllable AC/heat) in each room. Very unusual in-unit full-size washer and dryer. Two of the bedrooms have queen beds, and the third bedroom has a twin bed & trundle. The bedrooms are located in the back of the apartment, with the cozy living room, kitchen, and full-size dining table in the front facing the street and lovely homes of Flatbush.The spacePlease note, Flatbush is a vibrant city neighborhood in central Brooklyn. Visitors hoping to spend a significant time in Manhattan should expect around 30-40 minutes on the subway (Wall Street area) or 40-50 minutes (Times Square/Midtown). This space is also not particularly well-suited to people who need quiet. The top floor of the building is also occupied full-time by a family with several young children. Child-related noises should be expected!'

In [98]:
button_details = driver.page_source
button_details_clicked = BeautifulSoup(button_details)

In [231]:
detail_soup_clicked.get_text()

'Renovated Modern Apt in heart of Brooklyn! - Guest suites for Rent in Brooklyn, New York, United States - AirbnbSkip to contentWe’re sorry, some parts of the Airbnb website don’t work properly without JavaScript enabled.Start your searchPlaces to stayExperiencesLocationNavigate forward to access suggested resultsGo anywhere, anytimeI’m flexibleCheck inAdd datesCheck outAdd datesGuestsAdd guestsSearchBecome a HostRenovated Modern Apt in heart of Brooklyn!4.78 ·27 reviews·\U000f0003Superhost·Brooklyn, New York, United StatesShareSaveShow all photosPhotosAmenitiesReviewsLocation$133\xa0night$133 per night4.78 ·27 reviewsCheck availability Entire guest suite hosted by\xa0Saskia6 guests · 3 bedrooms · 4 beds · 2 bathsSelf check-inCheck yourself in with the lockbox.Saskia is a SuperhostSuperhosts are experienced, highly rated hosts who are committed to providing great stays for guests.Great check-in experience95% of recent guests gave the check-in process a 5-star rating.Located in the hear

In [224]:
def extract_review_element(listing_html, params):
    # 1. Find the right tag
    if 'class' in params:
        elements_found = listing_html.find_all(params['tag'], params['class'])
    else:
        elements_found = listing_html.find_all(params['tag'])

    # 2. Extract text from these tags
    if 'get' in params:
        element_texts = [el.get(params['get']) for el in elements_found]
    else:
        element_texts = [el.get_text() for el in elements_found]
    
    # 2. Extract the right element
    tag_order = params.get('order', 0)
    element = elements_found[tag_order]
        
    # 3. Select a particular text or concatenate all of them
    tag_order = params.get('order', 0)
    if tag_order == -1:
        output = '**__**'.join(element_texts)
    else:
        output = element_texts[tag_order]

    return output

In [226]:
extract_review_element(detail_soup_clicked, {'tag': 'span', 'class': 'll4r2nl dir dir-ltr', 'order':-1})

'Located in the heart of Brooklyn\'s "Little Haiti," this 1,000 sq ft 3 bedroom/2bath comes with all the amenities! Newly renovated, with splits (controllable AC/heat) in each room. Very unusual in-unit full-size washer and dryer. Two of the bedrooms have queen beds, and the third bedroom has a twin bed & trundle. The bedrooms are located in the back of the apartment, with the cozy living room, kitchen, and full-size dining table in the front facing the street and lovely homes of Flatbush.The spacePlease note, Flatbush is a vibrant city neighborhood in central Brooklyn. Visitors hoping to spend a significant time in Manhattan should expect around 30-40 minutes on the subway (Wall Street area) or 40-50 minutes (Times Square/Midtown). This space is also not particularly well-suited to people who need quiet. The top floor of the building is also occupied full-time by a family with several young children. Child-related noises should be expected!**__**Amazing stay in Brooklyn. Transportatio

In [677]:
def extract_soup_js(listing_url, waiting_time=[5, 1]):
    """Extracts HTML from JS pages: open, wait, click, wait, extract"""
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--blink-settings=imagesEnabled=false')
    driver = webdriver.Chrome('/Users/Vero/Downloads/chromedriver', options=options)
    

    driver.get(listing_url)
    time.sleep(waiting_time[0])
    
    reviews = 0
    try:
        element = driver.find_element_by_class_name('_koyt9le').click()
        reviews = 1
    except:
        pass # REVIEWS BUTTON NOT FOUND 
    
    # if the element is present - click on it
    if reviews == 1:
        for i in range(10): # 10 attempts to scroll to the price button
            try:
                actions = ActionChains(driver)
                driver.execute_script("arguments[0].scrollIntoView(true);", element);
                actions.move_to_element_with_offset(element, 5, 5)
                actions.click().perform()
                break
            except:
                pass
        
    driver.execute_script("window.scrollTo(0, 0);")
    
    time.sleep(waiting_time[1])
    detail_page = driver.page_source

    driver.quit()

    return BeautifulSoup(detail_page, features='html.parser')

In [228]:
hello = extract_soup_js(detail_url, waiting_time=[5, 1])

In [232]:
hello.find('span', 'll4r2nl dir dir-ltr')

<span class="ll4r2nl dir dir-ltr">Located in the heart of Brooklyn's "Little Haiti," this 1,000 sq ft 3 bedroom/2bath comes with all the amenities! Newly renovated, with splits (controllable AC/heat) in each room. Very unusual in-unit full-size washer and dryer. <br/><br/>Two of the bedrooms have queen beds, and the third bedroom has a twin bed &amp; trundle. The bedrooms are located in the back of the apartment, with the cozy living room, kitchen, and full-size dining table in the front facing the street and lovely homes of Flatbush.<br/><br/><span class="_1di55y9">The space</span><br/>Please note, Flatbush is a vibrant city neighborhood in central Brooklyn. Visitors hoping to spend a significant time in Manhattan should expect around 30-40 minutes on the subway (Wall Street area) or 40-50 minutes (Times Square/Midtown). <br/><br/>This space is also not particularly well-suited to people who need quiet. The top floor of the building is also occupied full-time by a family with several 

In [489]:
hello.find_all('a')

[<a class="screen-reader-only screen-reader-only-focusable skip-to-content" data-hook="skip-to-content" href="#site-content">Skip to content</a>,
 <a aria-label="Airbnb Homepage" class="_1y3vpg06" href="/"><div class="_1grlqto"><svg fill="currentcolor" height="32" style="display:block" width="102"><path d="M29.24 22.68c-.16-.39-.31-.8-.47-1.15l-.74-1.67-.03-.03c-2.2-4.8-4.55-9.68-7.04-14.48l-.1-.2c-.25-.47-.5-.99-.76-1.47-.32-.57-.63-1.18-1.14-1.76a5.3 5.3 0 00-8.2 0c-.47.58-.82 1.19-1.14 1.76-.25.52-.5 1.03-.76 1.5l-.1.2c-2.45 4.8-4.84 9.68-7.04 14.48l-.06.06c-.22.52-.48 1.06-.73 1.64-.16.35-.32.73-.48 1.15a6.8 6.8 0 007.2 9.23 8.38 8.38 0 003.18-1.1c1.3-.73 2.55-1.79 3.95-3.32 1.4 1.53 2.68 2.59 3.95 3.33A8.38 8.38 0 0022.75 32a6.79 6.79 0 006.75-5.83 5.94 5.94 0 00-.26-3.5zm-14.36 1.66c-1.72-2.2-2.84-4.22-3.22-5.95a5.2 5.2 0 01-.1-1.96c.07-.51.26-.96.52-1.34.6-.87 1.65-1.41 2.8-1.41a3.3 3.3 0 012.8 1.4c.26.4.45.84.51 1.35.1.58.06 1.25-.1 1.96-.38 1.7-1.5 3.74-3.21 5.95zm12.74 1.48a4

In [656]:
hello.find('class','_1e6wlg')

In [719]:
def process_detail_page(pd,url):
    soup = extract_soup_js(url, waiting_time=[3, 1])
    rules =  {'tag': 'span', 'class': 'll4r2nl dir dir-ltr', 'order':-1}
    
    features_list = pd
    features = extract_review_element(soup, rules)
    #Now get Description
    host_description = soup.find('span', 'll4r2nl dir dir-ltr')
    url = soup.find('a').get('href')
    cleaned_ids = ids.group(1).replace('?', '')
    data = {'Reviews': features,'Description': host_description}
    features_list = features_list.append(data, ignore_index=True)
    
    
    

    return features_list

In [721]:
processing = process_detail_page(ny_gen_300,'https://www.airbnb.com/rooms/52355596?adults=1&children=0&infants=0&check_in=2022-11-11&check_out=2022-11-13&previous_page_section_name=1000&federated_search_id=7395566c-595c-4f62-a023-9d4f0c516690')

In [722]:
processing

,URL,Amenities,Listing_id,Location,Logistics_URL,Name,Price,Ratings,Description,Reviews
0,https://www.airbnb.com/s/New-York--New-York--U...,1 guest**__**1 bedroom**__**1 bed**__**1 share...,listing_54179941,Private room in Manhattan,/rooms/54179941?previous_page_section_name=100...,A Cabin in the City,$57 per night,4.76 (25 reviews),NaN,NaN
1,https://www.airbnb.com/s/New-York--New-York--U...,1 guest**__**1 bedroom**__**1 bed**__**1 share...,listing_43819542,Private room in Brooklyn,/rooms/43819542?previous_page_section_name=100...,"Cozy, quiet room in artist's apartment",$44 per night,4.74 (82 reviews),NaN,NaN
2,https://www.airbnb.com/s/New-York--New-York--U...,1 guest**__**1 bedroom**__**1 bed**__**1 share...,listing_771430,Private room in Brooklyn,/rooms/771430?previous_page_section_name=1000&...,Massive Room With a Balcony!,$48 per night,4.85 (55 reviews),NaN,NaN
3,https://www.airbnb.com/s/New-York--New-York--U...,1 guest**__**1 bedroom**__**1 bed**__**4 share...,listing_42682643,Room in boutique hotel in Manhattan,/rooms/42682643?previous_page_section_name=100...,SSB 12. SINGLE SHARED BATHROOM,$59 per night,,NaN,NaN
4,https://www.airbnb.com/s/New-York--New-York--U...,1 guest**__**1 bedroom**__**1 bed**__**1 bath*...,listing_53346269,Private room in Upper West Side,/rooms/53346269?previous_page_section_name=100...,"Clean, quiet, safe room by Central Park",$55 per night,4.80 (30 reviews),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
296,https://www.airbnb.com/s/New-York--New-York--U...,2 guests**__**Studio**__**1 bed**__**1 bath**_...,listing_53770865,Entire rental unit in Fort Greene,/rooms/53770865?previous_page_section_name=100...,Brooklyn Fort Greene BAM Barclay NETS desk Tub TV,$67 per night,5.0 (9 reviews),NaN,NaN
297,https://www.airbnb.com/s/New-York--New-York--U...,1 guest**__**1 bedroom**__**1 bed**__**1 share...,listing_53972219,Private room in Jersey City,/rooms/53972219?previous_page_section_name=100...,Gaby’s place,$28 per night,4.71 (7 reviews),NaN,NaN
298,https://www.airbnb.com/s/New-York--New-York--U...,2 guests**__**1 bedroom**__**1 bed**__**1 bath...,listing_57874,Entire rental unit in Harlem,/rooms/57874?previous_page_section_name=1000&f...,The Brownstone-Luxury 1 Bd Apt/NYC,$160 per night,4.86 (321 reviews),NaN,NaN
299,NaN,1 guest**__**1 bedroom**__**1 bed**__**1 share...,listing_576631082749806426,Hostel room in Midtown Manhattan,/rooms/576631082749806426?previous_page_sectio...,Individual Cubicle Room in NYC/not shared/just...,$82 per night,4.44 (9 reviews),NaN,NaN


In [603]:
df = pd.DataFrame(processing, columns=['Description', 'Reviews'])

In [247]:
# CPU intensive process -> use multiprocessing :)
from multiprocessing import Pool

In [248]:
# typically we could set "n" to the number of cpu's
import os
os.cpu_count()

8

In [353]:
t0 = time.time()
detail_features = process_detail_page(detail_url)
print(time.time() - t0)

7.670641183853149


In [297]:
url_trial = build_urls('https://www.airbnb.com/s/New-York--New-York--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=calendar&place_id=ChIJOwg_06VPwokRYv534QaPC8g&query=New%20York%2C%20New%20York%2C%20United%20States&adults=1&search_type=filter_change&room_types%5B%5D=Entire%20home%2Fapt')

In [586]:
# try for one page
base_features = process_search_pages(url_trial[4:5], DATA_SEARCH)

In [587]:
base_features

[{'Listing_id': 'listing_25802121',
  'Name': 'Large studio near Central Park/Columbus Circle',
  'Price': '$90 per night',
  'Amenities': '3 guests**__**Studio**__**1 bed**__**1 bath**__**Wifi**__**Air conditioning',
  'Ratings': '4.89\xa0(9 reviews)',
  'Location': 'Entire rental unit in Midtown',
  'Logistics_URL': '/rooms/25802121?adults=1&previous_page_section_name=1000&federated_search_id=40b9eb38-f938-4387-9a69-0f0495d7fdaf'},
 {'Listing_id': 'listing_49909523',
  'Name': 'Perfect location for first time NYC travelers!!',
  'Price': '$141 per night',
  'Amenities': '2 guests**__**Studio**__**1 bed**__**1 bath**__**Wifi**__**Kitchen**__**Air conditioning',
  'Ratings': '4.74\xa0(42 reviews)',
  'Location': 'Entire rental unit in Midtown',
  'Logistics_URL': '/rooms/49909523?adults=1&previous_page_section_name=1000&federated_search_id=40b9eb38-f938-4387-9a69-0f0495d7fdaf'},
 {'Listing_id': 'listing_563674033221110944',
  'Name': 'New York Upper West Side',
  'Price': '$135 per nig

In [618]:
listings_urls = ['https://www.airbnb.com'+l['Logistics_URL'] for l in base_features]

In [619]:
listings_urls

['https://www.airbnb.com/rooms/25802121?adults=1&previous_page_section_name=1000&federated_search_id=40b9eb38-f938-4387-9a69-0f0495d7fdaf',
 'https://www.airbnb.com/rooms/49909523?adults=1&previous_page_section_name=1000&federated_search_id=40b9eb38-f938-4387-9a69-0f0495d7fdaf',
 'https://www.airbnb.com/rooms/563674033221110944?adults=1&previous_page_section_name=1000&federated_search_id=40b9eb38-f938-4387-9a69-0f0495d7fdaf',
 'https://www.airbnb.com/rooms/53888650?adults=1&previous_page_section_name=1000&federated_search_id=40b9eb38-f938-4387-9a69-0f0495d7fdaf',
 'https://www.airbnb.com/rooms/44049457?adults=1&previous_page_section_name=1000&federated_search_id=40b9eb38-f938-4387-9a69-0f0495d7fdaf',
 'https://www.airbnb.com/rooms/45457047?adults=1&previous_page_section_name=1000&federated_search_id=40b9eb38-f938-4387-9a69-0f0495d7fdaf',
 'https://www.airbnb.com/rooms/42326675?adults=1&previous_page_section_name=1000&federated_search_id=40b9eb38-f938-4387-9a69-0f0495d7fdaf',
 'https://

In [308]:
len(listings_urls)

20

In [347]:
listings_urls[1]

'https://www.airbnb.com/rooms/17093180?adults=1&previous_page_section_name=1000&federated_search_id=94cef9b3-4719-4a33-a84e-d46c2df2e842'

In [309]:
# check the ratio of empty values
def check_empty(features):
    # -2 as we have 2 prices (-1) and 2 specialties (-1)
    cnt, cnt_empty = -2, -2
    for listing in features:
        for key in listing[0]:
            cnt += 1
            if listing[0][key] == 'empty':
                cnt_empty += 1
    return cnt_empty/cnt

In [680]:
def read_all_details(url,start,end):
    all_listing_details = pd.DataFrame()
    url_list = build_urls(url)
    base_features = process_search_pages(url_list[start:end], DATA_SEARCH)
    listings_urls = ['http://www.airbnb.com'+l['Logistics_URL'] for l in base_features]
    for listing in listings_urls:
        indv_details = process_detail_page(listing)
        all_listing_details['Full_URL'] = listing
        all_listing_details = all_listing_details.append(indv_details, ignore_index=True )
        
        
    return all_listing_details
    
    

In [694]:
def read_all_details_csv(csv,start):
    all_listing_details = csv   
    listings_urls = [sth for sth in csv["Logistics_URL"]]
    for listing in listings_urls:
        indv_details = process_detail_page(listing)
        all_listing_details = all_listing_details.append(indv_details, ignore_index=True )
        print(listings_urls.index(listing))
        print('still working!')
       
        
    return all_listing_details

In [695]:
read_all_details_csv('ny_tribeca',16)

TypeError: string indices must be integers

In [558]:
#Batching, put away for now! 
# for n_pools in [8]:
#     with Pool(n_pools) as pool:
#         result = pool.map(process_detail_page, listings_urls[0:8])
#     pool.close()
#     pool.join()
#     print('hello!')

In [512]:
#NY_20 (4:5 )
ny_20 = read_all_details('https://www.airbnb.com/s/New-York--New-York--United-States/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&flexible_trip_lengths%5B%5D=weekend_trip&date_picker_type=calendar&query=New%20York%2C%20New%20York%2C%20United%20States&place_id=ChIJOwg_06VPwokRYv534QaPC8g&source=structured_search_input_header&search_type=autocomplete_click')

In [513]:
ny_20.to_csv('ny_20.csv')

In [637]:
#NY_100 (0:4)
ny_100 = read_all_details(NY_GENERAL_SEARCH_URL)

In [638]:
ny_100

,Full_URL,Description,Reviews
0,http://www.airbnb.com/rooms/17093180?previous_...,[Passing through New York City for a short vis...,Passing through New York City for a short visi...
1,http://www.airbnb.com/rooms/17093180?previous_...,[the room it is in a 2 Beds Apartment to be sh...,the room it is in a 2 Beds Apartment to be sha...
2,http://www.airbnb.com/rooms/17093180?previous_...,[Quiet room available in an artist's apartment...,Quiet room available in an artist's apartment....
3,http://www.airbnb.com/rooms/17093180?previous_...,"[The biggest room in Brooklyn, furnished with ...","The biggest room in Brooklyn, furnished with Q..."
4,http://www.airbnb.com/rooms/17093180?previous_...,"[Welcome to Hotel 17 Extended Stay Hotel., [],...",Welcome to Hotel 17 Extended Stay Hotel.With i...
...,...,...,...
75,http://www.airbnb.com/rooms/17093180?previous_...,[Clean and beautifully furnished studio apartm...,Clean and beautifully furnished studio apartme...
76,http://www.airbnb.com/rooms/17093180?previous_...,"[I am easy going, accessible and health profes...","I am easy going, accessible and health profess..."
77,http://www.airbnb.com/rooms/17093180?previous_...,[This space is shared with one person but you...,This space is shared with one person but you ...
78,http://www.airbnb.com/rooms/17093180?previous_...,[A charming one bedroom apartment in the heart...,A charming one bedroom apartment in the heart ...


In [644]:
ny_100_2 = read_all_details(NY_GENERAL_SEARCH_URL,5,9)

In [647]:
ny_80 = read_all_details(NY_GENERAL_SEARCH_URL,10,14)

In [648]:
ny_80.to_csv('ny_80.csv')

In [639]:
ny_100.to_csv('ny_100.csv')

In [646]:
ny_100_2.to_csv('ny_100_2.csv')

In [540]:
pd.merge(ny_20,ny_gen_300,on='URL',how='outer')

,URL,Description,Reviews,Amenities,Listing_id,Location,Name,Price,Ratings
0,https://www.airbnb.com/rooms/4195503?previous_...,[Newly furnished studio with new bathroom. Loc...,Newly furnished studio with new bathroom. Loca...,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.airbnb.com/rooms/4195503?previous_...,[Come experience unmatched luxury and hospital...,Come experience unmatched luxury and hospitali...,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.airbnb.com/rooms/4195503?previous_...,"[This studio is brand new, equipped with every...","This studio is brand new, equipped with everyt...",NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.airbnb.com/rooms/4195503?previous_...,[Furnished hardwood floor room with brand new ...,Furnished hardwood floor room with brand new f...,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.airbnb.com/rooms/4195503?previous_...,[This apartment is very conveniently located o...,This apartment is very conveniently located on...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
314,https://www.airbnb.com/s/New-York--New-York--U...,NaN,NaN,1 guest**__**1 bedroom**__**1 bed**__**1 share...,listing_605110928280940374,Private room in Brooklyn,Private Room with Exposed Bricks Wall,$68 per night,
315,https://www.airbnb.com/s/New-York--New-York--U...,NaN,NaN,1 guest**__**1 bedroom**__**1 bed**__**1 share...,listing_53972219,Private room in Jersey City,Gaby’s place,$28 per night,4.71 (7 reviews)
316,https://www.airbnb.com/s/New-York--New-York--U...,NaN,NaN,2 guests**__**1 bedroom**__**1 bed**__**2 shar...,listing_49520777,Private room in Jersey City,CONTACT HOST BEFORE BOOKING!,$40 per night,
317,https://www.airbnb.com/s/New-York--New-York--U...,NaN,NaN,1 guest**__**1 bedroom**__**1 bed**__**2 share...,listing_14485877,Private room in Brooklyn,I call my place Susan's Villa,$37 per night,4.74 (39 reviews)


In [640]:
ny_gen_300.to_csv('ny_gen_300.csv')

In [654]:
dr_chelsea = read_all_details('http://airbnb.com/s/Chelsea-New-York/homes?',0,19)

In [661]:
dr_chelsea

,Full_URL,Description,Reviews
0,http://www.airbnb.com/rooms/32355657?previous_...,[Passing through New York City for a short vis...,Passing through New York City for a short visi...
1,http://www.airbnb.com/rooms/32355657?previous_...,[the room it is in a 2 Beds Apartment to be sh...,the room it is in a 2 Beds Apartment to be sha...
2,http://www.airbnb.com/rooms/32355657?previous_...,[Everything you want to explore is right outsi...,Everything you want to explore is right outsid...
3,http://www.airbnb.com/rooms/32355657?previous_...,"[It’s no secret. Manhattan has the best, if no...","It’s no secret. Manhattan has the best, if not..."
4,http://www.airbnb.com/rooms/32355657?previous_...,[Clean private studio-style apartment/flat wit...,Clean private studio-style apartment/flat with...
...,...,...,...
295,http://www.airbnb.com/rooms/32355657?previous_...,[Elegant 2-bed 2-bath triplex condo with a lar...,Elegant 2-bed 2-bath triplex condo with a larg...
296,http://www.airbnb.com/rooms/32355657?previous_...,[BRAND NEW 2 bedroom apartment in the Times Sq...,BRAND NEW 2 bedroom apartment in the Times Squ...
297,http://www.airbnb.com/rooms/32355657?previous_...,[Entire 1-bedroom apartment literally a block ...,Entire 1-bedroom apartment literally a block a...
298,http://www.airbnb.com/rooms/32355657?previous_...,"[Located in the highly desirable West Village,...","Located in the highly desirable West Village, ..."


In [668]:
ny_chelsea = get_all_listings_data_300('http://airbnb.com/s/Chelsea-New-York/homes?')

In [669]:
ny_chelsea

,Full_URL,Amenities,Listing_id,Location,Logistics_URL,Name,Price,Ratings
0,http://airbnb.com/s/Chelsea-New-York/homes?&it...,1 guest**__**1 bedroom**__**1 bed**__**1 share...,listing_54179941,Private room in New York,/rooms/54179941?previous_page_section_name=100...,A Cabin in the City,$50 per night,4.76 (25 reviews)
1,http://airbnb.com/s/Chelsea-New-York/homes?&it...,1 guest**__**1 bedroom**__**1 bed**__**1 share...,listing_54146235,Private room in New York,/rooms/54146235?previous_page_section_name=100...,Micro Hostel in The Heart of Chelsea New York,$60 per night,4.77 (30 reviews)
2,http://airbnb.com/s/Chelsea-New-York/homes?&it...,2 guests**__**1 bedroom**__**1 bed**__**1 shar...,listing_41395190,Private room in New York,/rooms/41395190?previous_page_section_name=100...,spacious chelsea room,$60 per night,4.95 (66 reviews)
3,http://airbnb.com/s/Chelsea-New-York/homes?&it...,2 guests**__**1 bedroom**__**1 bed**__**1 priv...,listing_36239568,Room in boutique hotel in New York,/rooms/36239568?previous_page_section_name=100...,"Staypineapple New York, Fashionista King",$116 per night,4.70 (111 reviews)
4,http://airbnb.com/s/Chelsea-New-York/homes?&it...,4 guests**__**Studio**__**2 beds**__**1 bath**...,listing_4220151,Entire rental unit in New York,/rooms/4220151?previous_page_section_name=1000...,"Clean/Quiet Apt, Great Location, with washer/d...",$97 per night,4.45 (372 reviews)
...,...,...,...,...,...,...,...,...
295,http://airbnb.com/s/Chelsea-New-York/homes?&it...,4 guests**__**1 bedroom**__**1 bed**__**1 bath...,listing_27067725,Entire rental unit in New York,/rooms/27067725?previous_page_section_name=100...,Entire 1-bedroom Apartment near 34th st/5th Ave,$188 per night,4.76 (87 reviews)
296,http://airbnb.com/s/Chelsea-New-York/homes?&it...,2 guests**__**1 bedroom**__**1 bed**__**1 bath...,listing_30394467,Entire rental unit in New York,/rooms/30394467?previous_page_section_name=100...,"Chic Central Chelsea 1BR w/ Gym, Balcony, Door...",$440 per night,
297,http://airbnb.com/s/Chelsea-New-York/homes?&it...,4 guests**__**2 bedrooms**__**2 beds**__**1 ba...,listing_593667497500742005,Entire rental unit in New York,/rooms/593667497500742005?previous_page_sectio...,2BR Amazing Art haus in Chelsea! Mins to PENN ...,$347 per night,
298,http://airbnb.com/s/Chelsea-New-York/homes?&it...,6 guests**__**2 bedrooms**__**3 beds**__**1 ba...,listing_48589219,Entire rental unit in New York,/rooms/48589219?previous_page_section_name=100...,Big 2BR apartment with private patio in NYC!,$230 per night,4.67 (3 reviews)


In [663]:
dr_chelsea.to_csv('dr_chelsea.csv')

In [670]:
ny_chelsea.to_csv('ny_chelsea.csv')

In [741]:
#UPPER WEST SIDE 
ny_uws = get_all_listings_data_300('http://airbnb.com/s/Upper-West-Side-New-York/homes?')

In [742]:
ny_uws

,Full_URL,Amenities,Home_URL,Listing_id,Location,Name,Price,Ratings
0,http://airbnb.com/s/Upper-West-Side-New-York/h...,2 guests**__**Studio**__**1 bed**__**1 bath**_...,/rooms/45689161?previous_page_section_name=100...,listing_45689161,Entire rental unit in Upper West Side,Upper West Side Studio near Central Park/River...,$145 per night,5.0 (3 reviews)
1,http://airbnb.com/s/Upper-West-Side-New-York/h...,1 guest**__**1 bedroom**__**1 bed**__**1 bath*...,/rooms/53346269?previous_page_section_name=100...,listing_53346269,Private room in Upper West Side,"Clean, quiet, safe room by Central Park",$56 per night,4.80 (30 reviews)
2,http://airbnb.com/s/Upper-West-Side-New-York/h...,2 guests**__**Studio**__**1 bed**__**1 bath**_...,/rooms/51464921?previous_page_section_name=100...,listing_51464921,Entire condo in Upper West Side,Perfectly Designed Cozy Parkside Studio -Queen...,$174 per night,4.92 (26 reviews)
3,http://airbnb.com/s/Upper-West-Side-New-York/h...,2 guests**__**1 bedroom**__**1 bed**__**1 priv...,/rooms/28125509?previous_page_section_name=100...,listing_28125509,Private room in Upper West Side,Private bed/ bath in classic Upper West Side apt.,$110 per night,4.95 (135 reviews)
4,http://airbnb.com/s/Upper-West-Side-New-York/h...,2 guests**__**1 bedroom**__**2 beds**__**1 bat...,/rooms/53167175?previous_page_section_name=100...,listing_53167175,Entire rental unit in Upper East Side,East 80s stunner [maid + laundry included],$118 per night,
...,...,...,...,...,...,...,...,...
295,http://airbnb.com/s/Upper-West-Side-New-York/h...,2 guests**__**1 bedroom**__**1 bed**__**1 bath...,/rooms/53172095?previous_page_section_name=100...,listing_53172095,Entire rental unit in Harlem,"Light-filled, one-bedroom apartment Upper West...",$248 per night,
296,http://airbnb.com/s/Upper-West-Side-New-York/h...,4 guests**__**1 bedroom**__**4 beds**__**1 bat...,/rooms/19405371?previous_page_section_name=100...,listing_19405371,Entire rental unit in Harlem,Modern and Artful Home in Manhattan/NYC,$65 per night,4.87 (120 reviews)
297,http://airbnb.com/s/Upper-West-Side-New-York/h...,2 guests**__**1 bedroom**__**1 bed**__**1 bath...,/rooms/51008522?previous_page_section_name=100...,listing_51008522,Entire rental unit in Upper West Side,"Fantastic UWS 1BR w/ Elevator, Doorman, Gym, n...",$353 per night,
298,http://airbnb.com/s/Upper-West-Side-New-York/h...,6 guests**__**3 bedrooms**__**3 beds**__**2 ba...,/rooms/34336328?previous_page_section_name=100...,listing_34336328,Entire condo in Harlem,Gorgeous Central Park Penthouse,$299 per night,5.0 (7 reviews)


In [685]:
ny_uws.to_csv('ny_uws.csv')

In [682]:
dr_uws = read_all_details('http://airbnb.com/s/Upper-West-Side-New-York/homes?',0)

still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still working!
still work

TimeoutException: Message: timeout: Timed out receiving message from renderer: 600.000
  (Session info: headless chrome=100.0.4896.127)


In [691]:
uws_list = 'http://airbnb.com'+ ny_uws['Logistics_URL']
uws_list

0      http://airbnb.com/rooms/28125509?previous_page...
1      http://airbnb.com/rooms/53346269?previous_page...
2      http://airbnb.com/rooms/52996527?previous_page...
3      http://airbnb.com/rooms/45689161?previous_page...
4      http://airbnb.com/rooms/47783561?previous_page...
                             ...                        
295    http://airbnb.com/rooms/39859368?previous_page...
296    http://airbnb.com/rooms/19405371?previous_page...
297    http://airbnb.com/rooms/557481334642663609?pre...
298    http://airbnb.com/rooms/2563611?previous_page_...
299    http://airbnb.com/rooms/29331191?previous_page...
Name: Logistics_URL, Length: 300, dtype: object

In [743]:
def read_all_details_pd(pd,start):
    listings_urls = list(pd['Home_URL'])
    for listing in listings_urls:
        search_url = str('http://airbnb.com') + pd['Home_URL']
        indv_details = process_detail_page(pd,search_url)
        all_listing_details = all_listing_details.append(indv_details, ignore_index=True )
        print(listings_urls.index(search_url))
        print('still working!')
       
        
    return all_listing_details

In [697]:
ny_uws['Logistics_URL'] = str('http://airbnb.com') + ny_uws["Logistics_URL"]

In [744]:
read_all_details_pd(ny_uws,16)

TypeError: Object of type Series is not JSON serializable

In [683]:
dr_uws

,Full_URL,Description,Reviews
0,http://www.airbnb.com/rooms/8480087?previous_p...,[This apartment is professionally managed by F...,This apartment is professionally managed by Fu...
1,http://www.airbnb.com/rooms/8480087?previous_p...,[Furnished hardwood floor room (separated from...,Furnished hardwood floor room (separated from ...
2,http://www.airbnb.com/rooms/8480087?previous_p...,[This thoughtfully designed and bright studio ...,This thoughtfully designed and bright studio w...
3,http://www.airbnb.com/rooms/8480087?previous_p...,"[High End, modern, studio apartment on the Upp...","High End, modern, studio apartment on the Uppe..."
4,http://www.airbnb.com/rooms/8480087?previous_p...,"[Beautiful spacious historic, pre-WW1, high ce...","Beautiful spacious historic, pre-WW1, high cei..."
5,http://www.airbnb.com/rooms/8480087?previous_p...,[This room is very well located two blocks fro...,This room is very well located two blocks from...
6,http://www.airbnb.com/rooms/8480087?previous_p...,[Ideal live + work accommodation on storybook-...,Ideal live + work accommodation on storybook-l...
7,http://www.airbnb.com/rooms/8480087?previous_p...,[Big Bedroom with a Private Bathroom with a pr...,Big Bedroom with a Private Bathroom with a pri...
8,http://www.airbnb.com/rooms/8480087?previous_p...,"[Enjoy a stylish experience in high ceiling, r...","Enjoy a stylish experience in high ceiling, re..."
9,http://www.airbnb.com/rooms/8480087?previous_p...,[Lovely historical place in the safest area of...,Lovely historical place in the safest area of ...


In [740]:
ny_uws

,Full_URL,Amenities,Home_URL,Listing_id,Location,Name,Price,Ratings
0,http://airbnb.com/s/Upper-West-Side-New-York/h...,2 guests**__**1 bedroom**__**1 bed**__**1 priv...,http://airbnb.comhttp://airbnb.com/rooms/28125...,listing_28125509,Private room in Upper West Side,Private bed/ bath in classic Upper West Side apt.,$110 per night,4.95 (135 reviews)
1,http://airbnb.com/s/Upper-West-Side-New-York/h...,1 guest**__**1 bedroom**__**1 bed**__**1 bath*...,http://airbnb.comhttp://airbnb.com/rooms/53346...,listing_53346269,Private room in Upper West Side,"Clean, quiet, safe room by Central Park",$56 per night,4.80 (30 reviews)
2,http://airbnb.com/s/Upper-West-Side-New-York/h...,2 guests**__**1 bedroom**__**1 bed**__**1.5 sh...,http://airbnb.comhttp://airbnb.com/rooms/52996...,listing_52996527,Private room in Upper West Side,Very nice room in upper West side,$99 per night,4.73 (11 reviews)
3,http://airbnb.com/s/Upper-West-Side-New-York/h...,2 guests**__**Studio**__**1 bed**__**1 bath**_...,http://airbnb.comhttp://airbnb.com/rooms/45689...,listing_45689161,Entire rental unit in Upper West Side,Upper West Side Studio near Central Park/River...,$145 per night,5.0 (3 reviews)
4,http://airbnb.com/s/Upper-West-Side-New-York/h...,2 guests**__**1 bedroom**__**1 bed**__**1 priv...,http://airbnb.comhttp://airbnb.com/rooms/47783...,listing_47783561,Private room in Upper West Side,Big Room w/ private entrance and Private Bathr...,$91 per night,4.91 (43 reviews)
...,...,...,...,...,...,...,...,...
295,http://airbnb.com/s/Upper-West-Side-New-York/h...,2 guests**__**Studio**__**1 bed**__**1 bath**_...,http://airbnb.comhttp://airbnb.com/rooms/17122...,listing_17122439,Entire rental unit in Harlem,Charming Studio with your Own Private Patio,$111 per night,4.88 (227 reviews)
296,http://airbnb.com/s/Upper-West-Side-New-York/h...,3 guests**__**1 bedroom**__**2 beds**__**1 bat...,http://airbnb.comhttp://airbnb.com/rooms/12403...,listing_12403252,Entire rental unit in Harlem,Suite One at Bryant Manor,$195 per night,4.77 (200 reviews)
297,http://airbnb.com/s/Upper-West-Side-New-York/h...,1 guest**__**1 bedroom**__**1 bed**__**1 share...,http://airbnb.comhttp://airbnb.com/rooms/16412...,listing_16412823,Private room in Upper East Side,Beautiful Area. Central Park views. Classic Apt,$132 per night,4.89 (72 reviews)
298,http://airbnb.com/s/Upper-West-Side-New-York/h...,2 guests**__**1 bedroom**__**1 bed**__**1 bath...,http://airbnb.comhttp://airbnb.com/rooms/50372...,listing_50372478,Entire rental unit in Upper West Side,Central Park at Your Doorstep,$265 per night,


In [684]:
dr_uws.to_csv('dr_uws.csv')

In [700]:
read_all_details_pd(ny_uws,16)

WebDriverException: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: headless chrome=100.0.4896.127)
